installs

In [1]:
%%script echo skipping
%pip install snowflake-connector-python
%pip install pyarrow
%pip install pandas
!pip3 install confluent-kafka

skipping


imports

In [2]:
from kafka_helpers import *
from snowflake_helpers import *
from staging_helpers import get_raw_logs_df
from snowflake.connector.pandas_tools import write_pandas

/opt/homebrew/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (9.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


### **SETUP KAFKA**

In [3]:
consumer = connect_to_kafka_consumer()
data = stream_kafka_topic(consumer, KAFKA_TOPIC_NAME, 20)

In [4]:
data

['2022-10-05 14:42:19.238648 mendoza v9: [INFO]: Telemetry - hrt = 123; rpm = 34; power = 7.473425372\n',
 '2022-10-05 14:42:19.739315 mendoza v9: [INFO]: Ride - duration = 208.0; resistance = 30\n',
 '2022-10-05 14:42:20.239987 mendoza v9: [INFO]: Telemetry - hrt = 123; rpm = 33; power = 7.114899267999999\n',
 '2022-10-05 14:42:20.740665 mendoza v9: [INFO]: Ride - duration = 209.0; resistance = 30\n',
 '2022-10-05 14:42:21.241336 mendoza v9: [INFO]: Telemetry - hrt = 120; rpm = 34; power = 7.473425372\n',
 '2022-10-05 14:42:21.742015 mendoza v9: [INFO]: Ride - duration = 210.0; resistance = 30\n',
 '2022-10-05 14:42:22.242694 mendoza v9: [INFO]: Telemetry - hrt = 118; rpm = 35; power = 7.837601654\n',
 '2022-10-05 14:42:22.743383 mendoza v9: [INFO]: Ride - duration = 211.0; resistance = 30\n',
 '2022-10-05 14:42:23.244056 mendoza v9: [INFO]: Telemetry - hrt = 118; rpm = 34; power = 7.473425372\n',
 '2022-10-05 14:42:23.744739 mendoza v9: [INFO]: Ride - duration = 212.0; resistance = 3

### **PROCESS KAFKA W PANDAS**

In [5]:
raw_logs_df = get_raw_logs_df(data)
raw_logs_df

,log
0,2022-10-05 14:42:19.238648 mendoza v9: [INFO]:...
1,2022-10-05 14:42:19.739315 mendoza v9: [INFO]:...
2,2022-10-05 14:42:20.239987 mendoza v9: [INFO]:...
3,2022-10-05 14:42:20.740665 mendoza v9: [INFO]:...
4,2022-10-05 14:42:21.241336 mendoza v9: [INFO]:...
5,2022-10-05 14:42:21.742015 mendoza v9: [INFO]:...
6,2022-10-05 14:42:22.242694 mendoza v9: [INFO]:...
7,2022-10-05 14:42:22.743383 mendoza v9: [INFO]:...
8,2022-10-05 14:42:23.244056 mendoza v9: [INFO]:...
9,2022-10-05 14:42:23.744739 mendoza v9: [INFO]:...


In [6]:
raw_logs_df.dtypes

log    object
dtype: object

### **SETUP SNOWFLAKE**

connection, cursor, schemas

In [7]:
conn = connect_to_snowflake()
cs = conn.cursor()

Ensure cursor is set to use right schema

In [8]:
create_staging_schema(cs)
use_staging_schema(cs)

Check tables in schema

In [9]:
show_tables(cs, STAGING_SCHEMA)

[(datetime.datetime(2022, 10, 5, 4, 44, 45, 266000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'DF_TEST_TABLE',
  'DELATON',
  'YUSRA_STORIES_STAGING',
  'TABLE',
  '',
  '',
  2001,
  56320,
  'SYSADMIN',
  '1',
  'OFF',
  'OFF',
  'N'),
 (datetime.datetime(2022, 10, 5, 3, 29, 48, 451000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'DF_TEST_TABLE_2',
  'DELATON',
  'YUSRA_STORIES_STAGING',
  'TABLE',
  '',
  '',
  0,
  0,
  'SYSADMIN',
  '1',
  'OFF',
  'OFF',
  'N'),
 (datetime.datetime(2022, 10, 4, 4, 39, 6, 220000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'TEST_TABLE',
  'DELATON',
  'YUSRA_STORIES_STAGING',
  'TABLE',
  '',
  '',
  2,
  1024,
  'SYSADMIN',
  '1',
  'OFF',
  'OFF',
  'N')]

### **PUTTING DF INTO STAGING SCHEMA**

WRITE_PANDAS WAY 

In [10]:
raw_logs_df.columns.values

array(['log'], dtype=object)